# Data pre-processing
1. Using selective search to find proposed objects from each image
2. Using IoU to check if proposed object box overlaps with any of the real boxes and if they do, we mark them as positives samples, and if not, we mark them as negative samples.

In [339]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from utils.IoU import compute_iou

### Setting up paths here for original images and labels path

In [340]:
cwd = os.getcwd()

original_data_path = "original_data"
original_images_path = os.path.join(cwd, original_data_path, "images")
original_labels_path = os.path.join(cwd, original_data_path, "labels")

### Setting up paths here for proposed images and labels path

In [341]:
base_path = "proposed_dataset"
proposed_car = os.path.join(cwd, base_path, "car")
proposed_non_car = os.path.join(cwd, base_path, "non_car")

### Max proposed objects per image found using selective search

In [342]:
max_proposals = 5000
max_proposal_infer = 200

### Max number of positive and negative samples to be generated per image

In [343]:
max_positive = 30
max_negative = 10

In [344]:
input_dimension = (640, 480)
model_path = "car_detector.h5"

In [345]:
def selective_search(image, method="fast"):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    if method == "fast":
        ss.switchToSelectiveSearchFast()
    else:
        ss.switchToSelectiveSearchQuality()
    rects = ss.process()
    return rects

In [346]:
def load_image(image_path):
    """
    Load image from path and convert it to RGB and resize it to input dimension
    Parameters:
        image_path: path to image
    Returns:
        image: ndarray of shape (640, 480, 3)
    """
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, input_dimension)
    return image

In [347]:
def load_label(label_path):
    """
    Loads all the label information from the label file for one image. Makes dictionary with all the objects found in the file.
    Parameters:
        label_path: path to label
    Returns:
        label: dictionary
    """
    with open(label_path, "r") as f:
        lines = f.readlines()
    labels = {}
    for line in lines:
        line = line.strip().split(" ")
        width = int(float(line[3]) * input_dimension[0])
        height = int(float(line[4]) * input_dimension[1])
        x_min = int(float(line[1]) * input_dimension[0] - (width / 2))
        y_min = int(float(line[2]) * input_dimension[1] - (height / 2))
        x_max = int(x_min + width)
        y_max = int(y_min + height)
        labels[line[0]] = {
            "x_min": x_min,
            "y_min": y_min,
            "x_max": x_max,
            "y_max": y_max,
        }
    return labels

In [348]:
def save_image(image, image_path):
    """
    Saves image to path
    Parameters:
        image: ndarray of shape (640, 480, 3)
        image_path: path to save image
    """
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(image_path, image)

In [349]:
def compare_proposed_rect_to_one_label(rect, label):
    """
    Compares proposed object box to one label box and return True if IoU is greater than 0.7, false if IoU is less than 0.05 and None if IoU is between 0.05 and 0.7.
    Parameters:
        rect: tuple of (x_min, y_min, x_max, y_max)
        label: dictionary
    Returns:
        iou: float
    """
    IoU = compute_iou(label, rect)
    if IoU > 0.7:
        return True
    elif IoU < 0.05:
        return False
    else:
        return None

In [350]:
def check_all_rectangles(rect, labels, image):
    """
    Checks if proposed object box overlaps with any of the real boxes and if they do, we mark them as positives samples, and if not, we mark them as negative samples.
    Also saves all the positive and negative samples to folder.
    Parameters:
        rect: proposed object box
        labels: dictionary of real boxes
    Returns:
        label: 1 if positive sample, 0 if negative sample
    """
    positive_ROI = 0
    negative_ROI = 0
    global totalPositive
    global totalNegative

    for k, v in labels.items():
        real_box = [v["x_min"], v["y_min"], v["x_max"], v["y_max"]]
        value = compare_proposed_rect_to_one_label(rect, real_box)

        fullOverLap = rect[0] >= real_box[0]
        fullOverLap = fullOverLap and rect[1] >= real_box[1]
        fullOverLap = fullOverLap and rect[2] <= real_box[2]
        fullOverLap = fullOverLap and rect[3] <= real_box[3]

        if value is True and positive_ROI < max_positive:
            roi = image[rect[1] : rect[3], rect[0] : rect[2]]
            roi = cv2.resize(roi, input_dimension, interpolation=cv2.INTER_CUBIC)
            filename = f"{totalPositive}.jpg"
            output_path = os.path.join(proposed_car, filename)
            save_image(roi, output_path)
            positive_ROI += 1
            totalPositive += 1
        elif value is False and negative_ROI < max_negative and not fullOverLap: # TODO Fix negative_roi < max_negative not working, will save every image
            roi = image[rect[1] : rect[3], rect[0] : rect[2]]
            roi = cv2.resize(roi, input_dimension, interpolation=cv2.INTER_CUBIC)
            filename = f"{totalNegative}.jpg"
            output_path = os.path.join(proposed_non_car, filename)
            save_image(roi, output_path)
            negative_ROI += 1
            totalNegative += 1


In [351]:
def draw_image_with_box(image, box_coords):
    """
    Draws box on image
    Parameters:
        image: ndarray of shape (224, 224, 3)
        box_coords: tuple of (x, y, w, h)
    Returns:
        image: ndarray of shape (224, 224, 3)
    """
    x_min, y_min, x_max, y_max = box_coords
    image = cv2.rectangle(
        image, (x_min, y_max), (x_max, y_min), (0, 255, 0), 2
    )
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.show()

In [352]:
images_to_process = len(os.listdir(original_images_path))
print("Total images to process: ", images_to_process)

Total images to process:  46


In [353]:
totalPositive = 0
totalNegative = 0

for i, image_name in enumerate(os.listdir(original_images_path)[:]):
    print("Processing image: ", i + 1, "/", images_to_process)
    image_path = os.path.join(original_images_path, image_name)
    label_path = os.path.join(original_labels_path, image_name.split(".")[0] + ".txt")
    print("Image path: ", image_path)
    image = load_image(image_path)
    labels = load_label(label_path)
    rects = selective_search(image)
    positive_count = 0
    negative_count = 0
    print("Total proposed objects: ", len(rects))

    # draw_image_with_box(image, (labels["0"]["x_min"], labels["0"]["y_min"], labels["0"]["x_max"], labels["0"]["y_max"]))

    for rect in rects[:max_proposals]:
        x, y, w, h = rect
        x_min = x
        y_min = y
        x_max = x + w
        y_max = y + h
        proposed_box = [x_min, y_min, x_max, y_max]
        if x < 0 or y < 0 or w < 0 or h < 0:
            continue
        if x + w > image.shape[1] or y + h > image.shape[0]:
            continue
        check_all_rectangles(proposed_box, labels, image)

    print("-"*100)

Processing image:  1 / 46
Image path:  C:\Users\Alibaba\Desktop\deep_learning_project\deep_learning_project\R-CNN-test\original_data\images\0a22af19-frame27500.jpg
Total proposed objects:  1543
----------------------------------------------------------------------------------------------------
Processing image:  2 / 46
Image path:  C:\Users\Alibaba\Desktop\deep_learning_project\deep_learning_project\R-CNN-test\original_data\images\121e2adf-frame23000.jpg
Total proposed objects:  1422
----------------------------------------------------------------------------------------------------
Processing image:  3 / 46
Image path:  C:\Users\Alibaba\Desktop\deep_learning_project\deep_learning_project\R-CNN-test\original_data\images\1343e87c-frame14500.jpg
Total proposed objects:  1503
----------------------------------------------------------------------------------------------------
Processing image:  4 / 46
Image path:  C:\Users\Alibaba\Desktop\deep_learning_project\deep_learning_project\R-CNN-t